### Import and Device Configuration

In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
import torch
from copy import deepcopy
import timm
torch.set_printoptions(precision = 6, sci_mode = False)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
import sys
sys.path.append('../../pytei')

### Definition of Target Model

In [ ]:
test_model = timm.create_model('vit_tiny_patch16_224', pretrained = True, num_classes = 5)

### Error Injection using PyTEI

In [ ]:
from pytei import Injector

model = deepcopy(test_model).to(device)
model.eval()
test_input = torch.normal(size = (1, 3, 224, 224), mean = 0, std = 1).to(device)

with torch.no_grad():
    injector = Injector('./targets', p = 1e-4, device = device, verbose = True, mitigation = 'clip')
    print('----------Error free----------')
    print('Outputs (error-free):', model(test_input))

    print('----------Error Injected----------')
    injector.inject(model)
    print('Outputs (error-injected):', model(test_input))

    print('----------Error Mitigated----------')
    model = deepcopy(test_model).to(device)
    model.eval()
    injector.inject(model, use_mitigation = True)
    print('Outputs (error-mitigated):', model(test_input))

    injector.save_error_map('../../temp/testmap.pt', sparse = True)
    injector.load_error_map('../../temp/testmap.pt', sparse = True)